## Library import & define Func (for Crawling)

In [284]:
import sqlite3
import sqlalchemy

from requests import request, HTTPError
from requests.compat import urljoin, urlparse, quote, unquote
from time import sleep
import re
from urllib import parse

from requests.sessions import Session
import warnings
warnings.filterwarnings('ignore')

In [2]:
headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

In [75]:
def download(url, params={}, headers={}, method='GET', maximum=1): ## maxmum은 유한적인 loop를 위하여 필요
    
    resp = request(url = url,
                   params = params if method=='GET' else {},
                   data = params if method=='POST' else {},
                   headers = headers,
                   method = method)
    
    try:
        resp.raise_for_status()  # 200 OK 코드가 아닌 경우 에러 발동
    except HTTPError as e:
        if 500 <= e.response.status_code<600 and maximum > 0:
            print("RETRY")
            sleep(1)
            resp = download(url, paramas, headers, method, maximum-1)
        else:    
            print(e)
    
    
    
    return resp    

In [76]:
def can_fetch(url,path):
    resp = download(urljoin(url, '/robots.txt'))
    allow=[]
    disallow=[]
    
    if resp.status_code==200:
        useragent = '*' 
        for i in resp.text.splitlines():
            pair = i.split(':') ## key,value
            if len(pair) == 2:
                
                if pair[0].strip() == 'User-agent':
                    useragent = pair[1] 
                else:
                    if pair[0].strip() =='Allow':
                        allow.append(pair[1].strip())
                    else:
                        disallow.append(pair[1].strip())
    
    
    
    
    return True if path in allow else False, allow, disallow

In [77]:
def new_download(url):
    
    _ = urlparse(url)
    print(can_fetch(_[0]+'://'+_[1], _[2])[0]) ## True/ False를 print해주기
    
    return download(url,headers = headers)

In [6]:
def crawling(seed):
    resp = new_download(seed)
    
    ##추가작업=> content-type : text/*, application/json+xml, image
    dom = BeautifulSoup(resp.text, 'lxml')
    return [urljoin(seed, i.attrs['href']) for i in dom.select('a[href]') if not i.attrs['href'].startswith('#')] ## '#'
    ## '#'으로 시작하지 않는 a[href] 태그들의 url을 return 해주기

### naver mail의 쿠키 설정해주기

In [11]:
session = Session()
session.cookies.get_dict()

{}

- cookie의 row당 0-3번째 index만 쿠키로 설정해주기

In [12]:
## naver 쿠키 split해주기

s = 0
for i in [i for i,_ in enumerate(naver) if _ == 'Medium']:
    e=i
    
    session.cookies.set(naver[s], naver[s+1], domain=naver[s+2], path=naver[s+3]) ## 쿠키설정
    
    s = e+1

In [79]:
resp1 = download('https://mail.naver.com', headers=headers)


In [80]:
resp2 = session.get('https://mail.naver.com')


In [81]:
resp1.text==resp2.text

False

### Crawling  mail

In [23]:
'https://mail.naver.com/json/list/?page=1&sortField=1&sortType=0&folderSN=0&type=&isUnread=false&u=park11127'

'https://mail.naver.com/json/list/?page=1&sortField=1&sortType=0&folderSN=0&type=&isUnread=false&u=park11127'

- 받은메일함에서 request url 가져오기(받은 메일함 페이지 - 개발자도구 - 네트워크 - XHR - ?page... - headers - request url )

In [132]:
url='https://mail.naver.com/json/list'
params={
    'page':1,
    'sortField':1,
    'sortType':0,
    'folderSN':0,
    'type':None,
    'isUnread':'false',
    'u':'park11127'
}

In [21]:
resp = request('get', url=url, params=params)

In [22]:
resp.text

'{"Result":"FAIL","Message":"", "LoginStatus":"NOLOGIN"}'

In [24]:
resp2 = session.request('post', url=url, params=params)

In [28]:
resp.headers['Content-Type']

'text/plain;charset=UTF-8'

In [49]:
session.cookies.get_dict() ## session에 cookie가 잘 입력되었는지 확인해보기

{'ASID': '3d638b1c000001753ac0de0000000050',
 'MM_NEW': '1',
 'MM_NOW_COACH': '1',
 'NFS': '2',
 'NID_AUT': 'DOqGyiOBYfe0CZ9T3i75QYa/b+r5QKuyhBFcm0SJTLunpH3sxr3XcM1xNRnYe6Er',
 'NID_JKL': 'FSaBIjlk85JM7W7nQCv9GniOInhi4TGnbcTanfHIm38=',
 'NID_SES': 'AAABlzTvhiCYVogJCxvqm4/Ip0jt6wMpRAgfT90QpfAaHUR4Xx1OlEn8M8mD/DDLA4AYO0soUCyFy7OStQzTduTvPguhbV2VfOAbln084QbmYZRVtRTEPlYt+U/bdE0QiZX2k6+1vv4imPiKl6Z+zsTMGodUuKqJFKHXr9LlxdhUAvHZ7JtvraZ6xLutStkObobcEacAi8a9eb5Ttm0WyATHKA/wtSPBNXv7XQs70oUR/gxYFZFVlNc17n9cFw360eFcLamfzRpq2g6PMo+Ng+sGv5afzkagrmuV0uW44KTM/Q2rHUrUPghzzDbm7olRFOh3YriiKwdj8tvRBeDoOXxXgt3oT7PVeMb3w6ytwfNGHTD+8H+3JW/NZoVGPMkDDFJcl1vC0KSPdQw3dOURy2nEZMn2unSIlzljklFt0UFfXgxL7EPrzCvb4ex5tpL0tPsa5VN0J6w4i1ip9hZmrv6sMBdOEx5vN/ACykHhzA9ulMsXupy0Uwty5Z2dxXNvSi4N7jgj5KYP0a4EN7y3sg5tVo3v9H3jq/l+52Kn1Ph1zZPD',
 'NNB': 'IWGV6AVLTFCWA',
 'NRTK': 'ag#20s_gr#4_ma#0_si#2_en#2_sp#2',
 'NaverSuggestUse': 'use%26unuse',
 '_ga': 'GA1.2.1315744196.1608054668',
 '_ga_7VKFYR6RV1': 'GS1.1.1618297223.5.0.1618

In [38]:
for i,k in enumerate(resp.json()['mailData']): ## mail의 이름확인
    print(i+1,k['subject'])

1 축하드려요 데이터 전문가 포럼 (빅데이터분석기사, ADP, SQLP, DAP 등) 카페에 가입 되셨습니다.
2 [학생용]폭력예방교육 에 오신 것을 환영합니다.
3 [2021 빅데이터청년인재] 4조 - 아이디어
4 [RE]RE: 머신인텔리전스 연구실 학부 인턴 선발을 위한 인터뷰 일정 안내
5 [RE]Re: 머신인텔리전스 연구실 학부 인턴 선발을 위한 인터뷰 일정 안내
6 머신인텔리전스 연구실 학부 인턴 선발을 위한 인터뷰 일정 안내
7 [ICT인턴십] 국내과정 학생 서류지원 안내(~7/13(화), 13시까지)
8 [Webinar]: Accelerating Allegro PCB and APD Package Layout Using NVIDIA GPUs for Complex Designs
9 Re: 박근형 증빙서류 제출
10 sun dong eun님이 ‘강의자료’ 폴더를 공유했습니다.
11 데이터 청년 캠퍼스 관련 서류 및 사전 교육 정보 안내
12 Re: 제출 서류 관련 문의
13 AI·DATA INSIGHT 14호 - 기계번역 데이터셋 : 규모의 게임
14 [기업가정신 NOW] 로봇 스타트업 어떻게 육성해야 하나
15 데이터 청년 캠퍼스 출석체크 안내
16 고려대학교 청년캠퍼스 교육 안내
17 RE: RE: 오티관련메일 미수신
18 2021년 데이터 청년 캠퍼스 교육생 오리엔테이션 개최(비대면) 및 온라인 사전 교육 안내(고려대)
19 RE: 오티관련메일 미수신
20 사업계획서 작성방법_20210617.pdf
21 
	[JBNUHub 공지]
	[공지] 중간기말고사 성적
22 [RE]연구실 학부인턴 관련 궁금한 사항 문의드립니다
23 [네이버페이] 결제하신 내역을 안내해드립니다.
24 [한국화웨이] Seeds For The Future 2021 (글로벌 ICT 인재프로그램) 안내 및 참가 요청의 건 
25 고려대 데이터 청년 캠퍼스 최종선정 합격자 안내
26 Re: 201716175 박근형 2차 상담
27 Re: 교수님 2차상담 문의 드립니다
28 Re: 교수

## Selenium
- selenium은 웹사이트 테스트를 위한 도구로 브라우저 동작을 자동화할 수 있습니다. 셀레니움을 이용하는 웹크롤링 방식은 바로 이점을 적극적으로 활용하는 것입니다. 프로그래밍으로 브라우저 동작을 제어해서 마치 사람이 이용하는 것 같이 웹페이지를 요청하고 응답을 받아올 수 있습니다.
- 데이터 수집용이 아닌 메크로 만들때 쓰는 프레임 워크

In [50]:
! pip install selenium

In [85]:
import selenium
from selenium.webdriver import Chrome, ChromeOptions

In [86]:
selenium.__version__

'3.141.0'

In [63]:
driver = Chrome(r'C:/Users/chromedriver.exe')

In [87]:
url = 'https://www.naver.com'

In [88]:
driver.get(url) ## url로 접근

### dom탐색

In [93]:
from bs4 import BeautifulSoup

In [120]:
driver.find_element_by_css_selector('a.link_login').click() ## 로그인 링크를 찾아서 클릭 : 로그인 화면으로 전환


In [94]:
dom = BeautifulSoup(driver.page_source,'lxml')

In [98]:
for i in dom.find_all(text=re.compile(r'로그인')):
    print(i.find_parent())

<span>로그인 바로가기</span>
<a class="btn btn_login" data-clk="sug.cxlogin" href="https://nid.naver.com/nidlogin.login">로그인</a>
<h2 class="blind">로그인</h2>
<a class="link_login" data-clk="log_off.login" href="https://nid.naver.com/nidlogin.login?mode=form&amp;url=https%3A%2F%2Fwww.naver.com"><i class="ico_naver"><span class="blind">네이버</span></i>로그인</a>


In [105]:
for i in driver.find_elements_by_css_selector('form input[name]'): ## form 의 input tag(recursive=True)
    print(i.tag_name, i.get_attribute('name'))

input localechange
input dynamicKey
input encpw
input enctp
input svctype
input smart_LEVEL
input bvsd
input encnm
input locale
input url
input id
input pw
input nvlong


In [121]:
driver.find_element_by_name('id').clear() # clear
driver.find_element_by_name('pw').clear() # clear

driver.find_element_by_name('id').send_keys('park11127') ## id가 적혀진다.
driver.find_element_by_name('pw').send_keys('12345') ## pw가 적혀진다.

In [122]:
driver.find_element_by_css_selector('form input[type=submit]').get_attribute('value') ## 로그인 버튼

'로그인'

In [123]:
driver.find_element_by_css_selector('form input[type=submit]').click()

- 위에서 진행했던 쿠키 등록을 간편하게 할 수 있다.

In [126]:
naver = driver.get_cookies() ## login 후의 쿠키정보

In [127]:
session.cookies.clear()
session.cookies.get_dict()

{}

In [130]:
for i in naver:
    session.cookies.set(i['name'], i['value'], domain=i['domain'], path=i['path'])

In [131]:
session.cookies.get_dict() ## 쿠키 등록

{'NID_AUT': '0W9iJXmPKCdstwacE79HN/QXqjQ4TMoB+QjDTvH94qghty8F6fYILk86eDvE02Sk',
 'NID_JKL': 'Uy54vVp3sFDUUv5SI3qLzf7SaqW5RKKzTcTc9CC39B4=',
 'NID_SES': 'AAABkKzU/J2PxcREPEDB/wy6SRZvNCGkNXsDChVHIVA5GrkXjvnwLfm4y4Tw1jGj0n7Z5LyjHdMx3/HN64rTyPKaz+dN96jlcKnkcp0789w/wc/NYTupY4lsVQv6Gr8+SPYhA1+AlpxA3fVEALvlp3OhP7INoFt/I3exKCz/vTncVFIET49Tmd0bxfRU76DsbLzEu2QXDDD6naaiBfxFsIzmv/RmvZPEW2OSCHjjiclRyATn/zAhvMGMlnwphzHyHYIpKdmFQMrcKfaqhHPad31FYw4EQPcKs6lz6Wz8P/PO67SAR6uNkneBv025Y07guHJWqZvZ9w3p/TRSlAisuQsAFAs/Mt5Vri8ZXlRH8tEs+VKTQIxoxVDmMFtfreN2WS4fUktJ91h/iqOiaW4Thxwg/GeilGdZcXFcwrTJMUbQFj/avCtxQ0HhwjOk+154t0RRZy3aqypre+Phme4bM0BuFTvSY1TWGikIxVHuKxuTlu366Z+Q69457ISkYQuDHL4pzMbDsca5CxLum2r9qUVeKxU=',
 'NNB': 'SITZMUC6WHTWA',
 'nid_inf': '340511836',
 'JSESSIONID': 'F5C872283B951C4C4B28E0CC361591A8',
 'NM_THEME_EDIT': '',
 'NM_THEME_LAST_FIXED': 'HEALTH',
 'PM_CK_loc': '5b3cfba8eee6debe05f5d6bd00f0cae5df3615191d4c549fb193d9c04cc15e09'}

- url 과 param등록하여 메일함 확인해보기

In [133]:
url='https://mail.naver.com/json/list'
params={
    'page':1,
    'sortField':1,
    'sortType':0,
    'folderSN':0,
    'type':None,
    'isUnread':'false',
    'u':'park11127'
}

In [135]:
resp = session.request('post', url=url, params=params)

In [172]:
for i,k in enumerate(resp.json()['mailData']): ## mail의 이름확인
    print(i+1,k['subject'])
    print('='*50)

1 축하드려요 데이터 전문가 포럼 (빅데이터분석기사, ADP, SQLP, DAP 등) 카페에 가입 되셨습니다.
2 [학생용]폭력예방교육 에 오신 것을 환영합니다.
3 [2021 빅데이터청년인재] 4조 - 아이디어
4 [RE]RE: 머신인텔리전스 연구실 학부 인턴 선발을 위한 인터뷰 일정 안내
5 [RE]Re: 머신인텔리전스 연구실 학부 인턴 선발을 위한 인터뷰 일정 안내
6 머신인텔리전스 연구실 학부 인턴 선발을 위한 인터뷰 일정 안내
7 [ICT인턴십] 국내과정 학생 서류지원 안내(~7/13(화), 13시까지)
8 [Webinar]: Accelerating Allegro PCB and APD Package Layout Using NVIDIA GPUs for Complex Designs
9 Re: 박근형 증빙서류 제출
10 sun dong eun님이 ‘강의자료’ 폴더를 공유했습니다.
11 데이터 청년 캠퍼스 관련 서류 및 사전 교육 정보 안내
12 Re: 제출 서류 관련 문의
13 AI·DATA INSIGHT 14호 - 기계번역 데이터셋 : 규모의 게임
14 [기업가정신 NOW] 로봇 스타트업 어떻게 육성해야 하나
15 데이터 청년 캠퍼스 출석체크 안내
16 고려대학교 청년캠퍼스 교육 안내
17 RE: RE: 오티관련메일 미수신
18 2021년 데이터 청년 캠퍼스 교육생 오리엔테이션 개최(비대면) 및 온라인 사전 교육 안내(고려대)
19 RE: 오티관련메일 미수신
20 사업계획서 작성방법_20210617.pdf
21 
	[JBNUHub 공지]
	[공지] 중간기말고사 성적
22 [RE]연구실 학부인턴 관련 궁금한 사항 문의드립니다
23 [네이버페이] 결제하신 내역을 안내해드립니다.
24 [한국화웨이] Seeds For The Future 2021 (글로벌 ICT 인재프로그램) 안내 및 참가 요청의 건 
25 고려대 데이터 청년 캠퍼스 최종선정 합격자 안내
26 Re: 201716175 박근형 2차 상담
27 Re: 교수님 2차상담 문의 드립니다
28 Re: 교수

In [141]:
driver.delete_all_cookies() ## 드라이버 쿠키 지우기

In [142]:
driver.get('https://www.naver.com')

In [143]:
for i in naver:
    driver.add_cookie(i) ## 쿠키 추가해주기(naver)

In [144]:
driver.get('https://www.naver.com') ## refresh 해보면 로그인된 상태가 유지된다

In [145]:
driver.get('https://mail.naver.com') ## mail창으로 넘어가기

### dom 을 사용하여 mail함 접근

In [146]:
dom = BeautifulSoup(driver.page_source, 'lxml') ## dom 만들기

In [155]:
dom.find(text = re.compile('데이터 전문가 포럼')).find_parent().find_parent().find_parent().find_parent().find_parent() 

## mail리스트를 찾기위해 올라가보기

<ol class="mailList sender_context" style="min-width:600px"><li class="17954_li unmark _c1(mrCore|clickTitle|17954) _d2(mcDragndrop|html5DragStart)" data-category="203" draggable="true" foldersn="" mailsn="17954" style=""><span class="ico_current_flag"><i class="blind">마지막에 읽은 메일</i></span><ul class="mInfo"><li class="_m1(mlCore|toggleCheckMail|17954|down) _c1(mlCore|toggleCheckMail|17954|click) _stopDefault"><input class="mCheck _checkList _m3(mlCore|toggleCheckMail|17954|over) _m1(mlCore|toggleCheckMail|17954|down) _k1(mlCore|toggleCheckMail|17954|keydown) _stopDefault" id="mailCheck_17954" name="" onclick='ccr(this, "lst.selectgot", event)' resend="" securityleveltype="NORMAL" type="checkbox" value=""/><label class="blind" for="mailCheck_17954">"축하드려요 데이터 전문가 포럼 (빅데이터분석기사, ADP, SQLP, DAP 등) 카페에 가입 되셨습니다." 메일 선택</label></li><li class="important _m1(mlCore|doStop) _c1(mlCore|markMailSelf|17954|mark) _ccr(lst.mark) _stopDefault"><a href="#" onblur="$Element(this).removeClass('_starFocu

In [160]:
len(dom.select('.mailList > li')) ## 첫페이지에 뜨는 메일 개수

30

In [163]:
for i,k in enumerate(dom.select('.mailList > li li > label')):
    print(i+1, k.text)
    print('='*30)

1 "축하드려요 데이터 전문가 포럼 (빅데이터분석기사, ADP, SQLP, DAP 등) 카페에 가입 되셨습니다." 메일 선택
2 "[학생용]폭력예방교육 에 오신 것을 환영합니다." 메일 선택
3 "[2021 빅데이터청년인재] 4조 - 아이디어" 메일 선택
4 "[RE]RE: 머신인텔리전스 연구실 학부 인턴 선발을 위한 인터뷰 일정 안내" 메일 선택
5 "[RE]Re: 머신인텔리전스 연구실 학부 인턴 선발을 위한 인터뷰 일정 안내" 메일 선택
6 "머신인텔리전스 연구실 학부 인턴 선발을 위한 인터뷰 일정 안내" 메일 선택
7 "[ICT인턴십] 국내과정 학생 서류지원 안내(~7/13(화), 13시까지)" 메일 선택
8 "[Webinar]: Accelerating Allegro PCB and APD Package Layout Using NVIDIA GPUs for Complex Designs" 메일 선택
9 "Re: 박근형 증빙서류 제출" 메일 선택
10 "sun dong eun님이 ‘강의자료’ 폴더를 공유했습니다." 메일 선택
11 "데이터 청년 캠퍼스 관련 서류 및 사전 교육 정보 안내" 메일 선택
12 "Re: 제출 서류 관련 문의" 메일 선택
13 "AI·DATA INSIGHT 14호 - 기계번역 데이터셋 : 규모의 게임" 메일 선택
14 "[기업가정신 NOW] 로봇 스타트업 어떻게 육성해야 하나" 메일 선택
15 "데이터 청년 캠퍼스 출석체크 안내" 메일 선택
16 "고려대학교 청년캠퍼스 교육 안내" 메일 선택
17 "RE: RE: 오티관련메일 미수신" 메일 선택
18 "2021년 데이터 청년 캠퍼스 교육생 오리엔테이션 개최(비대면) 및 온라인 사전 교육 안내(고려대)" 메일 선택
19 "RE: 오티관련메일 미수신" 메일 선택
20 "사업계획서 작성방법_20210617.pdf" 메일 선택
21 "
	[JBNUHub 공지]
	[공지] 중간기말고사 성적" 메일 선택
22 "[RE]연구실 학부인턴 관련 궁금한 사항 문의드립니다" 메일 선택
23 "[네이버페이] 

### xpath를 활용하여 mail함 접근

/ ===> 자식, // ===> 자손,  [@파라미터='값'] <br>
ex) /div ==> root노드의 자식 div

In [169]:
driver.find_element_by_xpath('//ol') ## 자손중에 ol 찾기

<selenium.webdriver.remote.webelement.WebElement (session="1b59982aa664537724e9249d264b782f", element="c4fc0b13-ce12-418d-af5a-30a3625f9358")>

In [171]:
len(driver.find_elements_by_xpath('//ol/li//li/label')) ## 똑같이 메일을 찾아낸거 확인

30

### logout 해보기

In [221]:
driver.find_element_by_class_name('gnb_my').click() ## class gnb_my 클릭

In [215]:
logout = driver.find_element_by_xpath('//a[@id="gnb_logout_button"]') ## logout button을 id를 활용하여 찾기

In [216]:
logout.is_displayed(), logout.is_enabled(), logout.is_selected()

(True, True, False)

In [217]:
from selenium.webdriver.support import wait
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import ElementNotInteractableException 

In [218]:
wait = WebDriverWait(driver, 10, ignored_exceptions = [ElementNotInteractableException]) ## wait 객체는 ElementNotInteractableException오류를 무시하면서 10초동안 TRUE를 return할때까지 작동한다 

- logout

In [222]:
driver.find_element_by_class_name('gnb_my').click() ## class gnb_my 클릭
wait.until(lambda x : x.find_element_by_xpath('//a[@id="gnb_logout_button"]').is_displayed())
driver.find_element_by_xpath('//a[@id="gnb_logout_button"]').click()

### 팝업제거 이후 홈버튼 누르기(인터파크티켓 사이트)

In [224]:
driver.get('http://ticket.interpark.com/')

In [225]:
home = driver.find_element_by_css_selector('ul.gnbBtns > li > a:nth-of-type(1)')

In [226]:
home

<selenium.webdriver.remote.webelement.WebElement (session="1b59982aa664537724e9249d264b782f", element="5f271a3e-7a58-40c3-ab03-55557e7e8871")>

In [227]:
home.is_displayed(), home.is_enabled()

(True, True)

In [229]:
driver.implicitly_wait(10)
home.click()

ElementClickInterceptedException: Message: element click intercepted: Element <a href="javascript:;" class="btn_allMenu" onclick="allMenu()">...</a> is not clickable at point (20, 230). Other element would receive the click: <div class="noticeLayer">...</div>
  (Session info: chrome=91.0.4472.124)


In [231]:
from selenium.common.exceptions import ElementClickInterceptedException ## click과 관련된 에러 import

In [233]:
wait = WebDriverWait(driver, 10 , ignored_exceptions=[ElementClickInterceptedException,ElementNotInteractableException]) ## 새로운 wait객체 생성 

In [236]:
for i in driver.find_elements_by_xpath('//table[@class="noticeLayerTable"]'): ## table의 자손중 class가 noticeLayerTable인 것들 찾아주기===> 팝업창의 위치 찾아주기
    if i.is_displayed() and i.is_enabled(): ## 화면에 보여지고, 요소로도 보여질 경우에
        i.find_element_by_css_selector('div.closeDiv > a').click() ## 닫기 버튼 누르기

- try except 사용하기

In [238]:
from selenium.common.exceptions import StaleElementReferenceException 

In [244]:
from selenium.common.exceptions import StaleElementReferenceException

try:
    driver.find_element_by_css_selector('ul.gnbBtns > li > a:nth-of-type(1)').click()
except StaleElementReferenceException as e:
    for _ in driver.find_elements_by_xpath('//table[@class="noticeLayerTable"]'):
        if _.is_displayed() and _.is_enabled():
            _.find_element_by_css_selector('div.closeDiv > a').click()
    wait.until(lambda _:_.find_element_by_css_selector(
    'ul.gnbBtns > li > a:nth-of-type(1)').is_displayed())
    home.click()

### naver 검색

In [245]:
driver.get('http://www.naver.com')

In [246]:
for i in driver.find_elements_by_css_selector('form input'):
    print(i.get_attribute('name'))

sm
fbm
acr
acq
qdt
ie
acir
os
bid
pkid
eid
mra
query


In [262]:
driver.find_element_by_name('query').send_keys('파이썬') ## 검색창에 '파이썬' 작성하기
driver.find_element_by_id('search_btn').click() ##찾기 버튼 누르기 

In [265]:
for i in driver.find_elements_by_css_selector('.total_area > a'): 
    
    if urlparse(i.get_attribute('href')).netloc =='blog.naver.com':
        print(i.text, i.get_attribute('href')) 
        driver.get(i.get_attribute('href'))
        break  ## 블로그 주소를 찾아서 블로그링크로 들어가기
        

파이썬학원 수월한 과정! https://blog.naver.com/bredpark1004/222423051052


In [266]:
driver.implicitly_wait = 0

In [270]:
main = driver.find_element_by_css_selector('iframe')

- dom 에서 찾아보기

In [269]:
dom = BeautifulSoup(driver.page_source, 'lxml')
dom.select_one('iframe')

<iframe allowfullscreen="true" id="mainFrame" name="mainFrame" onload="oFramesetTitleController.start(self.frames['mainFrame'], self, sTitle);oFramesetTitleController.onLoadFrame();oFramesetUrlController.start(self.frames['mainFrame']);oFramesetUrlController.onLoadFrame()" scrolling="auto" src="/PostView.naver?blogId=bredpark1004&amp;logNo=222423051052&amp;redirect=Dlog&amp;widgetTypeCall=true&amp;directAccess=false"></iframe>

- dom을 스위칭시켜서 안에있는 dom을 해석하기 ===> 블로그 내용 가져오기

In [271]:
driver.switch_to_frame(main)

<ipython-input-271-0b4786396c90>:1: DeprecationWarning: use driver.switch_to.frame instead
  driver.switch_to_frame(main)


In [272]:
driver.find_element_by_css_selector('.se-main-container').text

'파이썬학원 수월한 과정!\n인생을 이렇게 살아도 되나 싶을 정도로, 목표가 없었습니다.\n이러한 사람들을 오늘만 산다고 하죠. 제가 딱 그랬습니다.\n 작년 한해는 어쩔 수 없이 허망하게 시간을 흘려 보냈지만 올해 만큼은 목표도 잡고, 원하는 공부도 해보면서 성취감을 느끼고 싶었는데요.\n어떤 걸 배울 지 여러 차례 고민을 해보다가, \n파이썬이라는 프로그래밍 언어를 알게 되었습니다.\n프로그래밍 언어는 요즘 직장인 분들도, \n대학생이나 고등학생! 심지어, 초등학생도 공부를 한다고 하더라고요.\n4차산업혁명으로 인해, 코딩, 프로그래밍, 개발자, AI 쪽의 전망이 좋아질 거라는 예상 때문이라고 생각하는데요. 저도 이왕 하는 공부! 노후까지 대비할 수 있다면 좋을 것 같아 파이썬 공부를 시작하게 되었습니다. \nPython은 프로그래밍 언어 중 하나이며, 이 밖에도 C언어나 C++, 자바 등의 다양한 언어들이 존재하는데요. 그 중에서도 python은 초보자들도 쉽게 배울 수 있는 수준이라고 하며, 프로그래밍의 기본이라고 하기에 경험이 없던 저도 자신감이 생겼습니다. \n독학으로 가능하다고 하더라도, 조금 불안했던 것이 사실인데요.\n괜히 혼자 했다가 어려움을 느끼고 금방 포기하게 될 것 같아, 차라리 처음부터 파이썬학원에 다니며 준비를 하는 게 좋을 것 같았습니다. \n그렇게 학원을 알아보게 되었고, 좋은 환경에서 지금도 잘 배우고 있는데요! Python을 배우고자 하는 분들을 위해 어떤 과정을 지나쳐 왔는 지 알려드리면 도움이 될 것 같아 글을 써보게 되었습니다. \n일단, 저는 IT 분야만을 전문적으로 가르치는 파이썬학원에서 배움을 시작했습니다. \n제가 수업을 들은 환경은 커리큘럼이 참 다양했는데요. 초심자인 저로서는 어떤 것부터 시작하면 될 지 몰라, 우선 멘토 선생님을 만났습니다. \n취업이나 이직 준비를 하는 분들은 멘토 선생님과 이야기를 나눈 후, 앞으로의 방향과 목표를 설정하는 편이라고 하는데요. 저는 이 때, 너무 사소한 걸로 만나 뵙게 된 

In [274]:
dom = BeautifulSoup(driver.page_source, 'lxml')
dom.select_one('.se-main-container').text

'\n\n\n\n\n\u200b\n\n\n\n \n\n\n\n파이썬학원 수월한 과정!\n\n\n\n\n\n\n\n\n\u200b인생을 이렇게 살아도 되나 싶을 정도로, 목표가 없었습니다.이러한 사람들을 오늘만 산다고 하죠. 제가 딱 그랬습니다. 작년 한해는 어쩔 수 없이 허망하게 시간을 흘려 보냈지만 올해 만큼은 목표도 잡고, 원하는 공부도 해보면서 성취감을 느끼고 싶었는데요.어떤 걸 배울 지 여러 차례 고민을 해보다가, 파이썬이라는 프로그래밍 언어를 알게 되었습니다.프로그래밍 언어는 요즘 직장인 분들도, 대학생이나 고등학생! 심지어, 초등학생도 공부를 한다고 하더라고요.\u200b\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\u200b4차산업혁명으로 인해, 코딩, 프로그래밍, 개발자, AI 쪽의 전망이 좋아질 거라는 예상 때문이라고 생각하는데요. 저도 이왕 하는 공부! 노후까지 대비할 수 있다면 좋을 것 같아 파이썬 공부를 시작하게 되었습니다. Python은 프로그래밍 언어 중 하나이며, 이 밖에도 C언어나 C++, 자바 등의 다양한 언어들이 존재하는데요. 그 중에서도 python은 초보자들도 쉽게 배울 수 있는 수준이라고 하며, 프로그래밍의 기본이라고 하기에 경험이 없던 저도 자신감이 생겼습니다. \u200b\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\u200b독학으로 가능하다고 하더라도, 조금 불안했던 것이 사실인데요.괜히 혼자 했다가 어려움을 느끼고 금방 포기하게 될 것 같아, 차라리 처음부터 파이썬학원에 다니며 준비를 하는 게 좋을 것 같았습니다. 그렇게 학원을 알아보게 되었고, 좋은 환경에서 지금도 잘 배우고 있는데요! Python을 배우고자 하는 분들을 위해 어떤 과정을 지나쳐 왔는 지 알려드리면 도움이 될 것 같아 글을 써보게 되었습니다. \u200b\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\u200b일단, 저는 IT 분야만을 전문적으로 가르치는 파이썬학원에서 배움을

- 다시 dom을 스위칭 해주기(원상태로)

In [275]:
driver.switch_to_default_content()

<ipython-input-275-1e095620afd7>:1: DeprecationWarning: use driver.switch_to.default_content instead
  driver.switch_to_default_content()


In [276]:
driver.find_element_by_css_selector('.se-main-container').text ### 다시 dom을 원상복귀 시키자 못찾는것 확인

KeyboardInterrupt: 

In [301]:
dom = BeautifulSoup(driver.page_source, 'lxml')
dom.select_one('.se-main-container').text

WebDriverException: Message: chrome not reachable
  (Session info: chrome=91.0.4472.124)


### window switching

In [277]:
driver.get('https://www.kt.com') ## 새창을 뛰워보자(빈창도 상관없음)

In [280]:
len(driver.window_handles) # === 새창을 뛰울경우 창이 2개라고 출력

2

In [285]:
driver.switch_to_window(driver.window_handles[-1]) ## 현재 윈도우가 아닌 다른 윈도우로 화면전환

===============================================
## Data Preprocessing(전처리)

- text 데이터들은 목적에 따라서 전처리방식이 달라진다(모델에 어울리는 데이터 수집도 중요(text))
- data project 의 중요한 부분

In [298]:
import nltk
import konlpy

In [300]:
from konlpy.tag import Okt

In [303]:
import os

In [312]:
len(os.listdir('./TXT')) ## 다운받은 뉴스기사 개수

112

### 어제 다운받은 뉴스기사 불러오기

In [330]:
with open('./TXT/'+os.listdir('./TXT')[1], 'r', encoding='utf-8') as f:
    data = f.read()

In [347]:
data

'오류를 우회하기 위한 함수 추가\n \n당국 "상황 악화 시 1. 7까지 올라가 하루 2천명 넘게 나올수도"어젯밤 기준으로 전국 생활치료센터 병상 34 남아서을 강북구 임시 선별진료소연합뉴스 자료사진\n서울연합뉴스 신재우 기자  수도권에서의 신종 코로나바이러스감염증코로나19 감염재생산지수가 1. 3 수준으로 높아진 것으로 나타났다.\n이상원 중앙방역대책본부 역학조사분석단장은 8일 정례브리핑에서 "현재 감염재생산지수는 전국이 1. 21에서 1. 29 정도이고, 수도권은 1. 25에서 1. 3 정도"라고 밝혔다.\n감염 재생산지수는 확진자 1명이 다른 사람 몇 명을 감염시키는지 나타내는 지표다. 이 지수가 1 미만이면 \'유행 억제\', 1 이상이면 \'유행 확산\'을 뜻한다.\n방대본은 이날 \'SEQIR\' 모델링 기법을 통해 코로나19 전파력이 현재의 강도로 유지될 경우 이달 말 하루 신규 확진자가 1천400명이 될 것으로 전망했다.\n상황이 악화할 경우에는 재생산지수가 1. 71로 올라가 하루 2천140명에 달하는 확진자가 나올 것으로 예상했다.\n수도권을 중심으로 코로나19 확진자가 속출하고 있어 병상 확보에도 비상이 걸렸다.\n중앙사고수습본부에 따르면 전날 오후 8시 기준으로 생활치료센터의 병상은 총 6천607개 가운데 2천242개33. 9가 남아있다. 정부와 광역시도는 환자 추가 발생을 대비해 생활치료센터 추가 지정을 서두르고 있다.\n수도권 생활치료센터의 병상 가동률은 서울 76. 1, 경기 78. 1, 인천 52. 7로 다른 지역보다 훨씬 높은 편이다.\n코로나19 재생산지수가 1. 3에 달해 당분간은 하루 1천명 이상의 확진자가 계속 발생할 것으로 보여 생활치료센터 확충이 시급한 상황이다.\n중증환자 치료를 위한 장비인력 등을 완비한 중환자 병상은 총 759개 중 585개77. 1가 남아있어 여유가 있는 편이다.\n감염병전담병원 내 확진자 입원 가능 병상도 총 7천405개 중 4천498개60. 7가 남아있고, 중증에서 상태가 호전되거나 중증으로

### 전처리 함수 만들기(정규식 이용)

In [350]:
def cleaning(data):
    data = re.sub(r'\b[a-z_][a-zA-Z_()]{4,}\b', '', data) ## 반드시 소문자+_ 1글자로 시작, 영문 4글자 이상인 패턴 삭제 ==>javascript부분 제거
    data = re.sub(r'\b[(){}/]{2,}','',data) ## 함수 기호 삭제 ==> () {} 제거
    data = re.sub(r'([.,?!])(\w)','\g<1> \g<2>', data) ## 구두점 다음에 글자가 있는 경우 공백 추가 ==> .단어 를 . 단어 로 변환 
    data = re.sub(r'\s{2,}', '\n', data) ## 공백 2개 이상인 경우 한개만 ==> \n\n\n\n\n ==> \n
    data = data.strip() ## 양쪽 공백 제거
    data = re.sub(r'[^\s\w\d.,?!:\'\"]', '', data) ## 공백 문자 숫자 구두점을 제외한 나머지 글자 삭제 (특수 문자 삭제)
    data = '\n'.join(data.splitlines()[1:]) ## 주석 삭제
    
    return data

###  전체 뉴스 전처리해서 저장하기

In [352]:
corpus=[]

for file in os.listdir('./TXT'):
    with open('./TXT/'+file, 'rb') as f:
        corpus.append(cleaning(f.read().decode('utf-8', errors='ignore')))

In [353]:
len(corpus)

112

### 전체 뉴스 data 하나로 합치기

In [355]:
data = ' '.join(corpus)

- sparse한 벡터표현(NLP)

In [362]:
len(os.listdir('./TXT')), len(data), len(data.split()), len(set(data.split())) ## unique 

(112, 125503, 28929, 13026)

===========================================================================
### 벡터 표현 방식
- count-based(one-hot vector, DTM) : frequency기반  ==>  숫자만 새면 된다
- co-occurence(word2vec, PMI) : context
- matrix-decomposition(SVD) 
- Prediction(LM) ===> BERT, ELMo

===========================================================================

### Count-Based
- 어떠한 단어의 빈도수가 높다 ==> 중요할 수 있지만 문법적인 역할을 하는 단어들이 자주 반복(빈도수가 높다고 중요한 단어가 꼭 아니다) 
- 즉 a, an, the ==> 조사 들은 주로 반복되지만 중요한 의미를 가지고 있지 않는다 ====> 삭제가능(불용어 삭제 ===> stopwords)

In [364]:
from collections import defaultdict

In [366]:
wc = defaultdict(int)   ## key error 를 방지하기 위함

In [367]:
for t in data.split():
    wc[t] += 1
    

In [370]:
sorted(wc.items(), key = lambda x : x[1], reverse=True)[:20] ## 빈도수가 높은 단어 20개 살펴보기(중요성이 있어보이진 않는다)

[('전', 242),
 ('있다.', 166),
 ('위한', 149),
 ('추가', 137),
 ('수', 117),
 ('오류를', 112),
 ('우회하기', 112),
 ('함수', 112),
 ('등', 108),
 ('이', 106),
 ('8일', 98),
 ('것으로', 91),
 ('지난', 87),
 ('이날', 83),
 ('한', 76),
 ('있는', 73),
 ('말했다.', 71),
 ('했다.', 70),
 ('김', 68),
 ('서울', 66)]

In [371]:
sorted(wc.items(), key = lambda x : x[1], reverse=False)[:20] ## 빈도수가 낮은 단어들(고유어들) ===> 여기서는 빈도수가 낮은 단어들의 중요성이 높아보이는 듯 ..

[('TV조선', 1),
 ('배틀', 1),
 ('국대다국민의힘', 1),
 ("대변인이다'에서", 1),
 ('4강전에', 1),
 ('진출한', 1),
 ('신인규왼쪽부터,', 1),
 ('김연주,', 1),
 ('임승호,', 1),
 ('양준우', 1),
 ('박경준', 1),
 ('방송인', 1),
 ('임백천', 1),
 ('배틀도', 1),
 ('생겼다.', 1),
 ('토론배틀인', 1),
 ("국대다'에는", 1),
 ('임', 1),
 ('부인인', 1),
 ('김연주', 1)]

In [377]:
sorted(wc.items(), key = lambda x : x[1], reverse=True)[5000:5020] ## 빈도수가 중간인 단어들 ==> 일반적으로 중요성이 크다

[('입원', 1),
 ('7천405개', 1),
 ('4천498개60.', 1),
 ('남아있고,', 1),
 ('중증에서', 1),
 ('상태가', 1),
 ('호전되거나', 1),
 ('중증으로', 1),
 ('환자의', 1),
 ('준중환자', 1),
 ('396개', 1),
 ('196개49.', 1),
 ('붐비는', 1),
 ('야간', 1),
 ('吳', 1),
 ('"좌파', 1),
 ('논리로', 1),
 ('미래세대', 1),
 ('지도자', 1),
 ('어려워""대한민국', 1)]

## n-gram

In [380]:
def ngram_umjeol(s, N=2): ## 2음절씩 끊어서 데이터 만들기
    rst = []
    for i in range(len(s)-1):
        rst.append(s[i:i+N])
    
    return rst

In [384]:
ngram_umjeol('금요일까지 오세요 여러분')

['금요', '요일', '일까', '까지', '지 ', ' 오', '오세', '세요', '요 ', ' 여', '여러', '러분']

### 2-gram

In [385]:
wc2 = defaultdict(int)

for i in data.split():
    for token in ngram_umjeol(i):
        wc2[token] += 1 

In [386]:
len(wc2.values()), len(wc2)

(12536, 12536)

In [393]:
sorted(wc2.items(), key = lambda x : x[1], reverse=True)[:20] ## 빈도수가 높은 단어 20개 살펴보기(2개씩 음절을 묶어서 데이터 전처리를 했을 경우)

[('다.', 1585),
 ('했다', 568),
 ('에서', 510),
 ('으로', 475),
 ('있다', 292),
 ('하고', 231),
 ('다고', 222),
 ('하는', 218),
 ('00', 210),
 ('다"', 198),
 ('이라', 193),
 ('지난', 181),
 ('20', 179),
 ('라고', 171),
 ('하기', 170),
 ('이다', 168),
 ('서울', 163),
 ('면서', 161),
 ('위한', 150),
 ('추가', 148)]

In [408]:
wc2['서울']

70

### 3-gram

In [396]:
wc3 = defaultdict(int)

for i in data.split():
    for token in ngram_umjeol(i, 3): ### 3-gram
        wc3[token] += 1 

In [399]:
sorted(wc3.items(), key = lambda x : x[1], reverse=True)[:20] ## 빈도수가 높은 단어 20개 살펴보기(2개씩 음절을 묶어서 데이터 전처리를 했을 경우)

[('다.', 1564),
 ('했다.', 494),
 ('으로', 445),
 ('에서', 440),
 ('다고', 219),
 ('하는', 207),
 ('하고', 194),
 ('있다.', 178),
 ('라고', 169),
 ('이다.', 163),
 ('면서', 152),
 ('위한', 150),
 ('하기', 149),
 ('추가', 140),
 ('다는', 130),
 ('코로나', 117),
 ('0.', 117),
 ('오류를', 112),
 ('류를', 112),
 ('우회하', 112)]

In [409]:
[i for i in wc3.keys() if i.startswith('서울')]

['서울',
 '서울연',
 '서울뉴',
 '서울시',
 '서울경',
 '서울본',
 '서울지',
 '서울의',
 '서울은',
 '서울에',
 '서울청',
 '서울과',
 '서울교',
 '서울성',
 '서울특',
 '서울,',
 '서울대',
 '서울소',
 "서울'",
 '서울만',
 '서울고']

In [410]:
keyword='서울'
candidate = [i for i in wc3.keys() if i.startswith(keyword)]

for i in candidate:
    print(i, wc3[i]/wc2[keyword]) ## 서울, 서울시 라는 단어가 빈도적 확률이 높다

서울 1.0
서울연 0.05714285714285714
서울뉴 0.3
서울시 0.2714285714285714
서울경 0.18571428571428572
서울본 0.02857142857142857
서울지 0.014285714285714285
서울의 0.04285714285714286
서울은 0.04285714285714286
서울에 0.04285714285714286
서울청 0.07142857142857142
서울과 0.02857142857142857
서울교 0.02857142857142857
서울성 0.014285714285714285
서울특 0.014285714285714285
서울, 0.014285714285714285
서울대 0.11428571428571428
서울소 0.014285714285714285
서울' 0.014285714285714285
서울만 0.014285714285714285
서울고 0.014285714285714285
